In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import kstest, norm
from scipy.stats import kurtosis, skew
from statsmodels.tsa.stattools import acf 
from tqdm import tqdm
import math
import torch.nn.functional as F

In [2]:
df1 = pd.read_csv("AMZN1.csv")  
df2 = pd.read_csv("AAPL.csv")  

df1['Date'] = pd.to_datetime(df1['Date'], dayfirst=True, errors='coerce')
df1 = df1[(df1['Date'].dt.year > 2010) & (df1['Date'].dt.year < 2021)]
df1 = df1.sort_values(by='Date')
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst=True, errors='coerce')
df2 = df2[(df2['Date'].dt.year > 2010) & (df2['Date'].dt.year < 2021)]
df2 = df2.sort_values(by='Date')

df1['Day'] = (df1['Date'] - df1['Date'].min()).dt.days
df1 = df1.sort_values(by = 'Day')
df1['log_return'] = np.log(df1['Adjusted Close']).diff()
df1.dropna(inplace=True) 
df2['Day'] = (df2['Date'] - df2['Date'].min()).dt.days
df2 = df2.sort_values(by = 'Day')
df2['log_return'] = np.log(df2['Adjusted Close']).diff()
df2.dropna(inplace=True) 

def make_log_return_df(a, n=1000, seed=42):
    
    np.random.seed(seed)
    mean = -a**2 / 2
    std = a
    log_returns = np.random.normal(loc=mean, scale=std, size=n)
    df = pd.DataFrame({'log_return': log_returns})
    return df

df1_fake = make_log_return_df(a = 0.5, n = 2000)
df2_fake = make_log_return_df(a = 0.05, n = 2000)

In [3]:
def prepare_lagged_data(df, col='log_return', num_lags=2):
    
    series = df[col].dropna().values
    N = len(series)

    X = np.stack([series[i:N - num_lags + i] for i in range(num_lags)], axis=1)
    y = series[num_lags:]

    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

X1, y1 = prepare_lagged_data(df1, col='log_return', num_lags=2)
X2, y2 = prepare_lagged_data(df2, col='log_return', num_lags=2)

X1_temp, X1_test, y1_temp, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42,shuffle=True)
X2_temp, X2_test, y2_temp, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42,shuffle=True)

X1_train, X1_val, y1_train, y1_val = train_test_split(X1_temp, y1_temp, test_size=0.25, random_state=42,shuffle=True)
X2_train, X2_val, y2_train, y2_val = train_test_split(X2_temp, y2_temp, test_size=0.25, random_state=42,shuffle=True)

print(f"\nShape of X1 (lagged inputs y_t): {X1.shape}") 
print(f"Shape of y1 (target r_{{t+1}}): {y1.shape}")
print(f"X1_train shape: {X1_train.shape}")
print(f"y1_train shape: {y1_train.shape}")
print(f"X1_val shape: {X1_val.shape}")
print(f"y1_val shape: {y1_val.shape}")
print(f"X1_test shape: {X1_test.shape}")
print(f"y1_test shape: {y1_test.shape}")

print(f"\nShape of X2 (lagged inputs y_t): {X2.shape}") 
print(f"Shape of y2 (target r_{{t+1}}): {y2.shape}")
print(f"X2_train shape: {X2_train.shape}")
print(f"y2_train shape: {y2_train.shape}")
print(f"X2_val shape: {X2_val.shape}")
print(f"y2_val shape: {y2_val.shape}")
print(f"X2_test shape: {X2_test.shape}")
print(f"y2_test shape: {y2_test.shape}")


Shape of X1 (lagged inputs y_t): torch.Size([2514, 2])
Shape of y1 (target r_{t+1}): torch.Size([2514])
X1_train shape: torch.Size([1508, 2])
y1_train shape: torch.Size([1508])
X1_val shape: torch.Size([503, 2])
y1_val shape: torch.Size([503])
X1_test shape: torch.Size([503, 2])
y1_test shape: torch.Size([503])

Shape of X2 (lagged inputs y_t): torch.Size([2514, 2])
Shape of y2 (target r_{t+1}): torch.Size([2514])
X2_train shape: torch.Size([1508, 2])
y2_train shape: torch.Size([1508])
X2_val shape: torch.Size([503, 2])
y2_val shape: torch.Size([503])
X2_test shape: torch.Size([503, 2])
y2_test shape: torch.Size([503])


In [4]:
def nll_loss(r_true, nu):
    var = nu**2 + 1e-8
    mean = -0.5 * var
    return 0.5 * torch.log(2 * torch.pi * var) + ((r_true - mean)**2) / (2 * var)

def compute_latent_z(r_true, nu):
    return (r_true + 0.5 * nu**2) / (nu + 1e-8)

def get_z(model, X, Y):
    X_t = torch.tensor(X, dtype=torch.float32)
    with torch.no_grad():
        sigma = model(X_t).squeeze().numpy()
    mu = -0.5 * sigma**2
    z = (Y - mu) / sigma
    return z
    
def compute_pred(model, X):
    with torch.no_grad():
        nu = model(X)
    nu = np.asarray(nu)
    z = np.random.normal(0, 1, size=nu.shape)
    return nu * z - 0.5 * nu**2

def summarize_distribution(data):
    data = np.asarray(data)
    
    # Basic stats
    mean = np.mean(data)
    std = np.std(data)
    skewness = skew(data)
    kurt = kurtosis(data, fisher=True)  # 0 = normal
    q25, q50, q75 = np.percentile(data, [25, 50, 75])

    # Print statistics
    print(f"  Mean      : {mean:.6f}")
    print(f"  Std Dev   : {std:.6f}")
    print(f"  Skewness  : {skewness:.6f}")
    print(f"  Kurtosis  : {kurt:.6f}")
    print(f"  Quantiles : 25%={q25:.4f}, 50%={q50:.4f}, 75%={q75:.4f}")


class RealizedVolNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        a = self.net(x)
        nu = torch.exp(a).squeeze()  # ν_t > 0
        return nu

In [5]:
def train_with_ks_tracking(X_train, y_train, X_val, y_val,
                           input_dim, lr=1e-3, epochs=500):
    model = RealizedVolNet(input_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Condition tracking
    consecutive_count = 0
    model_saved = False

    # Logs
    train_losses, val_losses, train_losses_ND = [], [], []
    train_pvalues, val_pvalues = [], []
    nus_train_ND = []
    nus_val = []
    for epoch in range(epochs):
        # === Train with dropout ===
        model.train()
        optimizer.zero_grad()
        nu_train = model(X_train)
        loss = torch.mean(nll_loss(y_train, nu_train))
        loss.backward()
        optimizer.step()

        # === Evaluate with dropout OFF ===
        model.eval()
        with torch.no_grad():
            nu_train_ND = model(X_train)
            nus_train_ND.append(torch.mean(nu_train_ND).item())
            loss_train_ND = torch.mean(nll_loss(y_train, nu_train_ND)).item()

            nu_val = model(X_val)
            nus_val.append(torch.mean(nu_val).item())
            loss_val = torch.mean(nll_loss(y_val, nu_val)).item()

            # Compute latent z for KS test
            z_train = compute_latent_z(y_train, nu_train_ND).cpu().numpy()
            z_val = compute_latent_z(y_val, nu_val).cpu().numpy()

            ks_train = kstest(z_train, 'norm').pvalue
            ks_val = kstest(z_val, 'norm').pvalue

        # === Record metrics ===
        train_losses.append(loss.item())
        train_losses_ND.append(loss_train_ND)
        val_losses.append(loss_val)
        train_pvalues.append(ks_train)
        val_pvalues.append(ks_val)
        
        if (epoch+1 ) % 50 == 0:
            print(f"Epoch {epoch:03d}: "
              f"Train NLL = {loss.item():.4f}, "
              f"Train ND NLL = {loss_train_ND:.4f}, "
              f"Val NLL = {loss_val:.4f}, "
              f"KS p (train) = {ks_train:.4f}, "
              f"KS p (Val) = {ks_val:.4f}")

        if epoch > 300:
            if loss_train_ND < loss_val:
                consecutive_count += 1
                if consecutive_count >= 10 and not model_saved:
                    print(f"\nSaving model at epoch {epoch} "
                          f"(train_loss_ND < test_loss for 5 consecutive epochs)")
                    print(f"Epoch {epoch:03d}: "
                      f"Train NLL = {loss.item():.4f}, "
                      f"Train ND NLL = {loss_train_ND:.4f}, "
                      f"Val NLL = {loss_val:.4f}, "
                      f"KS p (train) = {ks_train:.4f}, "
                      f"KS p (Val) = {ks_val:.4f}")
                    torch.save(model.state_dict(), "model_trainND_better_than_test3.pth")
                    model_saved = True
            else:
                consecutive_count = 0

    return model, {
        'train_losses': train_losses,
        'train_losses_ND': train_losses_ND,
        'val_losses': val_losses,
        'train_pvalues': train_pvalues,
        'val_pvalues': val_pvalues,
        'nus_train_ND': nus_train_ND,
        'nus_val' : nus_val
    }


In [6]:
model1, history1 = train_with_ks_tracking(X1_train, y1_train, X1_val, y1_val,
                                        input_dim=X1_train.shape[1], lr=1e-3, epochs=200)

Epoch 049: Train NLL = -2.1475, Train ND NLL = -2.2420, Val NLL = -2.2296, KS p (train) = 0.0000, KS p (Val) = 0.0000
Epoch 099: Train NLL = -2.4590, Train ND NLL = -2.5217, Val NLL = -2.4811, KS p (train) = 0.0000, KS p (Val) = 0.0000
Epoch 149: Train NLL = -2.4267, Train ND NLL = -2.5274, Val NLL = -2.4830, KS p (train) = 0.0000, KS p (Val) = 0.0000
Epoch 199: Train NLL = -2.4384, Train ND NLL = -2.5276, Val NLL = -2.4812, KS p (train) = 0.0000, KS p (Val) = 0.0000


In [7]:
model2, history2 = train_with_ks_tracking(X2_train, y2_train, X2_val, y2_val,
                                        input_dim=X2_train.shape[1], lr=1e-3, epochs=200)

Epoch 049: Train NLL = -1.1902, Train ND NLL = -1.2719, Val NLL = -1.2700, KS p (train) = 0.0000, KS p (Val) = 0.0000
Epoch 099: Train NLL = -2.4985, Train ND NLL = -2.6109, Val NLL = -2.6125, KS p (train) = 0.0000, KS p (Val) = 0.0000
Epoch 149: Train NLL = -2.5093, Train ND NLL = -2.5976, Val NLL = -2.5978, KS p (train) = 0.0000, KS p (Val) = 0.0000
Epoch 199: Train NLL = -2.4797, Train ND NLL = -2.6082, Val NLL = -2.6084, KS p (train) = 0.0000, KS p (Val) = 0.0000


In [8]:
z1 = get_z(model1, X1, y1)
z2 = get_z(model2, X2, y2)
rho = np.corrcoef(z1, z2)[0,1]
print(f"Estimated innovation correlation: {rho:.3f}")

Estimated innovation correlation: 0.433


C:\Users\adity\AppData\Local\Temp\ipykernel_59192\3399346040.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_t = torch.tensor(X, dtype=torch.float32)


In [11]:
num_lags = 2
init_lags_1 = df1['log_return'].dropna().values[-num_lags:]
init_lags_2 = df2['log_return'].dropna().values[-num_lags:]

def simulate_joint_path(model1, model2, start_lags1, start_lags2, n_steps, rho):
    path1 = list(start_lags1)
    path2 = list(start_lags2)
    num_lags = len(start_lags1)
    cov = np.array([[1.0, rho], [rho, 1.0]])
    
    for _ in range(n_steps):
        # Sample joint innovations
        z = np.random.multivariate_normal([0, 0], cov)
        
        # Prepare lag window tensors
        x1 = torch.tensor(path1[-num_lags:], dtype=torch.float32).unsqueeze(0)
        x2 = torch.tensor(path2[-num_lags:], dtype=torch.float32).unsqueeze(0)
        
        with torch.no_grad():
            sigma1 = model1(x1).item()
            sigma2 = model2(x2).item()
        
        r1 = -0.5 * sigma1 ** 2 + sigma1 * z[0]
        r2 = -0.5 * sigma2 **d 2 + sigma2 * z[1]
        
        path1.append(r1)
        path2.append(r2)
    
    return np.array(path1), np.array(path2)

def batch_simulate_joint_paths(model1, model2, X1, X2, n_steps, rho, n_mc=1):

    all_sim_r1 = []
    all_sim_r2 = []
    for i in range(X1.shape[0]):
        start1 = X1[i].numpy() if hasattr(X1[i], 'numpy') else X1[i]
        start2 = X2[i].numpy() if hasattr(X2[i], 'numpy') else X2[i]
        for _ in range(n_mc):
            sim_r1, sim_r2 = simulate_joint_path(model1, model2, start1, start2, n_steps, rho)
            # remove the initial lags, only keep simulated future
            all_sim_r1.append(sim_r1[len(start1):])
            all_sim_r2.append(sim_r2[len(start2):])
    return np.stack(all_sim_r1), np.stack(all_sim_r2)

##############################################################################

n_steps = 3        # Length of each simulated path
n_mc = 1            # Number of simulations per lagged window

sim_y1, sim_y2 = batch_simulate_joint_paths(model1, model2, X1, X2, n_steps, rho, n_mc=n_mc)

sim_y1 = sim_y1.flatten()
sim_y2 = sim_y2.flatten()

In [14]:
print("Real return correlation:", np.corrcoef(y1[:len(y2)], y2)[0,1])
print("Simulated return correlation:", np.corrcoef(sim_y1[num_lags:], sim_y2[num_lags:])[0,1])

Real return correlation: 0.4423793186554475
Simulated return correlation: 0.4491410629651049


# Using one of the dataset we are going to hedge the payoff of a 5 day call option 

In [40]:
def returns_to_spot_path(S_init, sim_returns):
    growth_factors = np.exp(sim_returns)
    return S_init * np.cumprod(np.concatenate([[1], growth_factors]))

def sim_paths_hedge(n_steps, start_lags1, start_lags2, S1_0,S2_0):
    sim_path1, sim_path2 = simulate_joint_path(model1, model2, start_lags1, start_lags2, n_steps, rho)
    sim_returns1 = sim_path1[2:]
    sim_spot1_path = returns_to_spot_path(S1_0, sim_returns1)
    sim_returns2 = sim_path2[2:]
    sim_spot2_path = returns_to_spot_path(S2_0, sim_returns2)
    return sim_spot1_path,sim_spot2_path

S1_0 = df1['Adjusted Close'].iloc[-6]
S2_0 = df2['Adjusted Close'].iloc[-6]
start_lags1 = [y1[-7], y1[-6]]  
start_lags2 = [y2[-7], y2[-6]]  
n_steps = 5

# Own RL Hedging

In [84]:
class BasketHedgingEnv:
    def __init__(self, S1_path, S2_path, strike, w1=0.5, w2=0.5, cost=0.0, include_prev_hedge=True):
        self.S1_path = S1_path
        self.S2_path = S2_path
        self.strike = strike
        self.n_steps = len(S1_path) - 1
        self.cost = cost
        self.w1 = w1
        self.w2 = w2
        self.include_prev_hedge = include_prev_hedge
        self.reset()

    def _get_state(self):
        S1_norm = self.S1 / self.S1_path[0]
        S2_norm = self.S2 / self.S2_path[0]
        time_norm = (self.n_steps - self.t) / self.n_steps
        if self.include_prev_hedge:
            state = [S1_norm, S2_norm, self.hedge1, self.hedge2, time_norm]
        else:
            state = [S1_norm, S2_norm, time_norm]
        return np.array(state, dtype=np.float32)

    def reset(self):
        self.t = 0
        self.S1 = self.S1_path[self.t]
        self.S2 = self.S2_path[self.t]
        self.hedge1 = 0.0
        self.hedge2 = 0.0
        self.cash = 0.0
        self.history = [(self.S1, self.S2, self.hedge1, self.hedge2, self.cash)]
        return self._get_state()

    def step(self, action):
        prev_hedge1, prev_hedge2 = self.hedge1, self.hedge2
        delta_hedge1 = action[0] - prev_hedge1
        delta_hedge2 = action[1] - prev_hedge2

        
        self.cash -= delta_hedge1 * self.S1
        self.cash -= delta_hedge2 * self.S2

       
        self.cash -= abs(delta_hedge1) * self.S1 * self.cost
        self.cash -= abs(delta_hedge2) * self.S2 * self.cost

        
        self.hedge1, self.hedge2 = action
        self.t += 1
        done = (self.t == self.n_steps)
        self.S1 = self.S1_path[self.t]
        self.S2 = self.S2_path[self.t]
        self.history.append((self.S1, self.S2, self.hedge1, self.hedge2, self.cash))

        state = self._get_state()
        reward = 0.0
        if done:
            
            self.cash += self.hedge1 * self.S1 + self.hedge2 * self.S2
            basket = self.w1 * self.S1 + self.w2 * self.S2
            payoff = max(basket - self.strike, 0)
            reward = -(self.cash - payoff)  
            self.history.append((self.S1, self.S2, 0.0, 0.0, self.cash))
        return state, reward, done, {}


In [85]:
import torch
import numpy as np

n_episodes = 10000  
strike = 140
all_rewards = []
optimizer = optim.Adam(policy.parameters(), lr=0.01)


for ep in range(n_episodes):
   
    start_lags1 = [y1[-7], y1[-6]]  
    start_lags2 = [y2[-7], y2[-6]]  
    S1_0 = df1['Adjusted Close'].iloc[-6] 
    S2_0 = df2['Adjusted Close'].iloc[-6]
   
    S1_path, S2_path = sim_paths_hedge(n_steps, start_lags1, start_lags2, S1_0, S2_0)
    
    include_prev_hedge = False  

    env = BasketHedgingEnv(S1_path, S2_path, strike, w1=0.5, w2=0.5, cost=0.0001, include_prev_hedge=include_prev_hedge)

    if include_prev_hedge:
        n_inputs = 5  
    else:
        n_inputs = 3  

    policy = BasketPolicyNet(n_inputs=n_inputs, hidden=64)

    # 4. Rollout episode
    state = torch.tensor(env.reset(), dtype=torch.float32)
    log_probs = []
    rewards = []
    done = False

    while not done:
        action = policy(state)
        action_np = np.clip(action.detach().numpy(), -2, 2)  
        state_new, reward, done, _ = env.step(action_np)
        log_prob = -((policy(state) - torch.tensor(action_np)).pow(2)).sum()
        log_probs.append(log_prob)
        rewards.append(reward)
        state = torch.tensor(state_new, dtype=torch.float32)
    R = sum(rewards)
    all_rewards.append(R)
    loss = -torch.stack(log_probs).sum() * R
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if ep % 1000 == 0:
        print(f"Episode {ep}, mean reward (last 50): {np.mean(all_rewards[-1000:]):.4f}")

    if ep % 1000 == 0:
        print(f"Episode {ep}, mean reward (last 50): {np.mean(all_rewards[-50:]):.4f}")
        print("Time | S1   | S2   | hedge1 | hedge2 | cash")
        for t, (s1, s2, h1, h2, cash) in enumerate(env.history):
            print(f"{t:>4} | {s1:>5.2f} | {s2:>5.2f} | {h1:>6.3f} | {h2:>6.3f} | {cash:>7.2f}")


Episode 0, mean reward (last 50): 3.4025
Episode 0, mean reward (last 50): 3.4025
Time | S1   | S2   | hedge1 | hedge2 | cash
   0 | 159.26 | 129.41 |  0.000 |  0.000 |    0.00
   1 | 154.91 | 126.66 |  0.793 | -0.349 |  -81.04
   2 | 156.53 | 126.53 |  0.750 | -0.354 |  -73.90
   3 | 161.75 | 130.52 |  0.713 | -0.373 |  -65.72
   4 | 160.04 | 129.76 |  0.680 | -0.400 |  -56.88
   5 | 158.38 | 127.98 |  0.638 | -0.402 |  -49.86
   6 | 158.38 | 127.98 |  0.000 |  0.000 |   -0.22
Episode 1000, mean reward (last 50): 5.1244
Episode 1000, mean reward (last 50): 5.1113
Time | S1   | S2   | hedge1 | hedge2 | cash
   0 | 159.26 | 129.41 |  0.000 |  0.000 |    0.00
   1 | 156.63 | 128.88 | -0.082 | -0.299 |   51.74
   2 | 164.31 | 133.69 | -0.051 | -0.248 |   40.35
   3 | 167.67 | 135.17 | -0.003 | -0.205 |   26.69
   4 | 165.40 | 135.49 |  0.045 | -0.171 |   14.00
   5 | 164.57 | 139.39 |  0.089 | -0.144 |    3.08
   6 | 164.57 | 139.39 |  0.000 |  0.000 |   -2.33
Episode 2000, mean reward (l